```
Image Classification : Implementation in Capsule Network - MNIST Datasets
Published Date: 15 October, 2018

author: Mohammed Innat
email:  innat1994@gmail.com
website: https://iphton.github.io/iphton.github.io/


Please feel free to use and modify this, but keep the above information. Thanks!

```

---


# MNIST Implementation Using Capsule Network

**CapsNets** are a new neural net architecture that may well have a profound impact on deep learning, in particular for computer vision. **Geoffrey Hinton** proposed a architecture that introduced a completely new type of neural network based on so-called **capsules**. 

Geoffrey Hinton - [Paper: Dynamic Routing Between Capsules](https://arxiv.org/abs/1710.09829). I found [this](https://www.oreilly.com/ideas/introducing-capsule-networks) blog post by **Aurélien Géron** well explained on this topic. However, most of the implemented function to build **CapsuleNet** is adopted from [Xifeng Guo Ph.D.](https://github.com/XifengGuo).

---

**About Data:**

The data files `train.csv` and `test.csv` contain gray-scale images of hand-drawn digits, from zero through nine.

Each image is `28` pixels in height and `28` pixels in width, for a total of `784` pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

The training data set, (**train.csv**), has `785` columns. The first column, called `label`, is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.

The test data set, (**test.csv**), is the same as the training set, except that it does not contain the `label` column.

Find On Kaggle: [Digit Recognizer](https://www.kaggle.com/c/digit-recognizer/data)

---

**Procedure:**
- **2. Data Preprocessing**
- **3. Building a Model**
- **5. Evaluate the Model**

**Miscellaneous:**
- Save Model and Weight 
- Tensorboard: Visualize the Computational Graph and Parameters.

Finally, Create `csv` file for kaggle submission.

---

**Result:**

I got **Final accuracy: 0.99** by implementing **Capsule Network** on **GeForce GTX 1050 Ti**. I set 10 epochs on the training process and took 40 mins. However, Training on a single CPU, epochs size should be set within < 3.

---

Let's begin with importing packages and dataset.

In [1]:
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)

from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
from keras.utils.vis_utils import plot_model
from keras import initializers, layers
import tensorflow as tf

import keras.backend as K

class Length(layers.Layer):
    """
    Compute the length of vectors. This is used to compute a Tensor that has the same shape with y_true in margin_loss
    inputs: shape=[dim_1, ..., dim_{n-1}, dim_n]
    output: shape=[dim_1, ..., dim_{n-1}]
    """
    def call(self, inputs, **kwargs):
        return K.sqrt(K.sum(K.square(inputs), -1))

    def compute_output_shape(self, input_shape):
        return input_shape[:-1]

class Mask(layers.Layer):
    """
    Mask a Tensor with shape=[None, d1, d2] by the max value in axis=1.
    Output shape: [None, d2]
    """
    def call(self, inputs, **kwargs):
        # use true label to select target capsule, shape=[batch_size, num_capsule]
        if type(inputs) is list:  # true label is provided with shape = [batch_size, n_classes], i.e. one-hot code.
            assert len(inputs) == 2
            inputs, mask = inputs
        else:  # if no true label, mask by the max length of vectors of capsules
            x = inputs
            # Enlarge the range of values in x to make max(new_x)=1 and others < 0
            x = (x - K.max(x, 1, True)) / K.epsilon() + 1
            mask = K.clip(x, 0, 1)  # the max value in x clipped to 1 and other to 0

        # masked inputs, shape = [batch_size, dim_vector]
        inputs_masked = K.batch_dot(inputs, mask, [1, 1])
        return inputs_masked

    def compute_output_shape(self, input_shape):
        if type(input_shape[0]) is tuple:  # true label provided
            return tuple([None, input_shape[0][-1]])
        else:
            return tuple([None, input_shape[-1]])


def squash(vectors, axis=-1):
    """
    The non-linear activation used in Capsule. It drives the length of a large vector to near 1 and small vector to 0
    :param vectors: some vectors to be squashed, N-dim tensor
    :param axis: the axis to squash
    :return: a Tensor with same shape as input vectors
    """
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm)
    return scale * vectors


class CapsuleLayer(layers.Layer):
    """
    The capsule layer. It is similar to Dense layer. Dense layer has `in_num` inputs, each is a scalar, the output of the 
    neuron from the former layer, and it has `out_num` output neurons. CapsuleLayer just expand the output of the neuron
    from scalar to vector. So its input shape = [None, input_num_capsule, input_dim_vector] and output shape = \
    [None, num_capsule, dim_vector]. For Dense Layer, input_dim_vector = dim_vector = 1.
    
    :param num_capsule: number of capsules in this layer
    :param dim_vector: dimension of the output vectors of the capsules in this layer
    :param num_routings: number of iterations for the routing algorithm
    """
    def __init__(self, num_capsule, dim_vector, num_routing=3,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_vector = dim_vector
        self.num_routing = num_routing
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

    def build(self, input_shape):
        assert len(input_shape) >= 3, "The input Tensor should have shape=[None, input_num_capsule, input_dim_vector]"
        self.input_num_capsule = input_shape[1]
        self.input_dim_vector = input_shape[2]

        # Transform matrix
        self.W = self.add_weight(shape=[self.input_num_capsule, self.num_capsule, self.input_dim_vector, self.dim_vector],
                                 initializer=self.kernel_initializer,
                                 name='W')

        # Coupling coefficient. The redundant dimensions are just to facilitate subsequent matrix calculation.
        self.bias = self.add_weight(shape=[1, self.input_num_capsule, self.num_capsule, 1, 1],
                                    initializer=self.bias_initializer,
                                    name='bias',
                                    trainable=False)
        self.built = True

    def call(self, inputs, training=None):
        # inputs.shape=[None, input_num_capsule, input_dim_vector]
        # Expand dims to [None, input_num_capsule, 1, 1, input_dim_vector]
        inputs_expand = K.expand_dims(K.expand_dims(inputs, 2), 2)

        # Replicate num_capsule dimension to prepare being multiplied by W
        # Now it has shape = [None, input_num_capsule, num_capsule, 1, input_dim_vector]
        inputs_tiled = K.tile(inputs_expand, [1, 1, self.num_capsule, 1, 1])

     
        # Compute `inputs * W` by scanning inputs_tiled on dimension 0. This is faster but requires Tensorflow.
        # inputs_hat.shape = [None, input_num_capsule, num_capsule, 1, dim_vector]
        inputs_hat = tf.scan(lambda ac, x: K.batch_dot(x, self.W, [3, 2]),
                             elems=inputs_tiled,
                             initializer=K.zeros([self.input_num_capsule, self.num_capsule, 1, self.dim_vector]))
        
        # Routing algorithm V2. Use iteration. V2 and V1 both work without much difference on performance
        assert self.num_routing > 0, 'The num_routing should be > 0.'
        for i in range(self.num_routing):
            c = tf.nn.softmax(self.bias, dim=2)  # dim=2 is the num_capsule dimension
            # outputs.shape=[None, 1, num_capsule, 1, dim_vector]
            outputs = squash(K.sum(c * inputs_hat, 1, keepdims=True))

            # last iteration needs not compute bias which will not be passed to the graph any more anyway.
            if i != self.num_routing - 1:
                # self.bias = K.update_add(self.bias, K.sum(inputs_hat * outputs, [0, -1], keepdims=True))
                self.bias += K.sum(inputs_hat * outputs, -1, keepdims=True)
            # tf.summary.histogram('BigBee', self.bias)  # for debugging
        return K.reshape(outputs, [-1, self.num_capsule, self.dim_vector])

    def compute_output_shape(self, input_shape):
        return tuple([None, self.num_capsule, self.dim_vector])


def PrimaryCap(inputs, dim_vector, n_channels, kernel_size, strides, padding):
    """
    Apply Conv2D `n_channels` times and concatenate all capsules
    :param inputs: 4D tensor, shape=[None, width, height, channels]
    :param dim_vector: the dim of the output vector of capsule
    :param n_channels: the number of types of capsules
    :return: output tensor, shape=[None, num_capsule, dim_vector]
    """
    output = layers.Conv2D(filters=dim_vector*n_channels, kernel_size=kernel_size, strides=strides, padding=padding)(inputs)
    outputs = layers.Reshape(target_shape=[-1, dim_vector])(output)
    return layers.Lambda(squash)(outputs)

Using TensorFlow backend.


In [2]:
from keras import layers, models
from keras import backend as K
from keras.utils import to_categorical
import keras

def CapsNet(input_shape, n_class, num_routing):
    """
    A Capsule Network on MNIST.
    :param input_shape: data shape, 4d, [None, width, height, channels]
    :param n_class: number of classes
    :param num_routing: number of routing iterations
    :return: A Keras Model with 2 inputs and 2 outputs
    """
    x = layers.Input(shape=input_shape)

    # Layer 1: Just a conventional Conv2D layer
    conv1 = layers.Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x)

    # Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_vector]
    primarycaps = PrimaryCap(conv1, dim_vector=8, n_channels=32, kernel_size=9, strides=2, padding='valid')

    # Layer 3: Capsule layer. Routing algorithm works here.
    digitcaps = CapsuleLayer(num_capsule=n_class, dim_vector=16, num_routing=num_routing, name='digitcaps')(primarycaps)

    # Layer 4: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
    # If using tensorflow, this will not be necessary. :)
    out_caps = Length(name='out_caps')(digitcaps)

    # Decoder network.
    y = layers.Input(shape=(n_class,))
    masked = Mask()([digitcaps, y])  # The true label is used to mask the output of capsule layer.
    x_recon = layers.Dense(512, activation='relu')(masked)
    x_recon = layers.Dense(1024, activation='relu')(x_recon)
    x_recon = layers.Dense(784, activation='sigmoid')(x_recon)
    x_recon = layers.Reshape(target_shape=[28, 28, 1], name='out_recon')(x_recon)

    # two-input-two-output keras Model
    return models.Model([x, y], [out_caps, x_recon])

def margin_loss(y_true, y_pred):
    """
    Margin loss for Eq.(4). When y_true[i, :] contains not just one `1`, this loss should work too. Not test it.
    :param y_true: [None, n_classes]
    :param y_pred: [None, num_capsule]
    :return: a scalar loss value.
    """
    L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))

    return K.mean(K.sum(L, 1))

# define model
model = CapsNet(input_shape=[28, 28, 1],
                n_class=10,
                num_routing=3)
model.summary()

Instructions for updating:
dim is deprecated, use axis instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 20, 20, 256)  20992       input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 6, 6, 256)    5308672     conv1[0][0]                      
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1152, 8)      0           conv2d_1[0][0]                   
______________________________________________

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.read_csv("Data/train.csv")
test = pd.read_csv("Data/test.csv")

Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 


print('Digit Counter:\n',Y_train.value_counts())
print('-'*30)
print('Check for Missing values:\n',X_train.isnull().any().describe() , '\n')
print('-'*30)
print('Check for Missing values:\n',test.isnull().any().describe() , '\n')
print('-'*30)

# Normalize the data
X_train = X_train / 255.0

# Reshape image in 3 dimensions 
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1, 28, 28, 1).astype('float32') / 255.

from keras.utils.np_utils import to_categorical 
Y_train = to_categorical(Y_train, num_classes = 10)


# Set the random seed
random_seed = 101

# Randomly split the data sets
from sklearn.model_selection import train_test_split
# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1,
                                                  random_state=random_seed)


def train(model, data, epoch_size_frac=1.0):
    """
    Training a CapsuleNet
    :param model: the CapsuleNet model
    :param data: a tuple containing training and testing data, like `((x_train, y_train), (x_test, y_test))`
    :param args: arguments
    :return: The trained model
    """
    # unpacking the data
    (x_train, y_train), (x_val, y_val) = data

    # callbacks
    log = callbacks.CSVLogger('CapsuleNet/log.csv')
    checkpoint = callbacks.ModelCheckpoint('CapsuleNet/CapsuleNet Weights Each Epochs/weights-{epoch:02d}.h5',
                                           save_best_only=True, save_weights_only=True, verbose=1)
    lr_decay = callbacks.LearningRateScheduler(schedule = lambda epoch: 0.001 * np.exp(-epoch / 10.))

    # compile the model
    model.compile(optimizer='adam',
                  loss=[margin_loss, 'mse'],
                  loss_weights=[1., 0.0005],
                  metrics={'out_caps': 'accuracy'})
    
    # Using the Tensorboard callback of Keras. 
    # https://stackoverflow.com/questions/42112260/how-do-i-use-the-tensorboard-callback-of-keras
    tbCallBack = keras.callbacks.TensorBoard(log_dir='CapsuleNet/CapGraph', histogram_freq=0, 
                                         write_graph=True, write_images=True)


    # -----------------------------------Begin: Training with data augmentation -----------------------------------#
    def train_generator(x, y, batch_size, shift_fraction=0.):
        train_datagen = ImageDataGenerator(width_shift_range = shift_fraction,
                                           height_shift_range = shift_fraction)  # shift up to 2 pixel for MNIST
        generator = train_datagen.flow(x, y, batch_size=batch_size)
        while 1:
            x_batch, y_batch = generator.next()
            yield ([x_batch, y_batch], [y_batch, x_batch])

    # Training with data augmentation. 
    model.fit_generator(generator=train_generator(x_train, y_train, 64, 0.1),
                        steps_per_epoch=int(epoch_size_frac*y_train.shape[0] / 64),
                        epochs = 10,
                        validation_data = [[x_val, y_val], [y_val, x_val]],
                        callbacks = [log, checkpoint, lr_decay, tbCallBack])
    # -----------------------------------End: Training with data augmentation -----------------------------------#


    return model

Digit Counter:
 1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: label, dtype: int64
------------------------------
Check for Missing values:
 count       784
unique        1
top       False
freq        784
dtype: object 

------------------------------
Check for Missing values:
 count       784
unique        1
top       False
freq        784
dtype: object 

------------------------------


In [4]:
%%time
train(model=model, data=((X_train, Y_train), (X_val, Y_val)), 
      epoch_size_frac = 0.5) # do 10% of an epoch (takes too long)

Epoch 1/10
295/295 [==============================] - 240s 813ms/step - loss: 0.1776 - out_caps_loss: 0.1775 - out_recon_loss: 0.1660 - out_caps_acc: 0.8103 - val_loss: 0.0395 - val_out_caps_loss: 0.0395 - val_out_recon_loss: 0.0703 - val_out_caps_acc: 0.9752

Epoch 00001: val_loss improved from inf to 0.03953, saving model to CapsuleNet/weights-01.h5
Epoch 2/10
295/295 [==============================] - 238s 807ms/step - loss: 0.0480 - out_caps_loss: 0.0480 - out_recon_loss: 0.0617 - out_caps_acc: 0.9668 - val_loss: 0.0314 - val_out_caps_loss: 0.0313 - val_out_recon_loss: 0.0677 - val_out_caps_acc: 0.9848

Epoch 00002: val_loss improved from 0.03953 to 0.03136, saving model to CapsuleNet/weights-02.h5
Epoch 3/10
295/295 [==============================] - 239s 810ms/step - loss: 0.0338 - out_caps_loss: 0.0338 - out_recon_loss: 0.0608 - out_caps_acc: 0.9771 - val_loss: 0.0256 - val_out_caps_loss: 0.0256 - val_out_recon_loss: 0.0670 - val_out_caps_acc: 0.9860

Epoch 00003: val_loss impro

In [5]:
def test(model, data):
    x_test, y_test = data
    y_pred, x_recon = model.predict([x_test, y_test], batch_size=100)
    print('Tesing....')
    print('-'*20)
    print('Test acc:', np.sum(np.argmax(y_pred, 1) == np.argmax(y_test, 1))/y_test.shape[0])

# call test function    
test(model = model, data = ( X_val[:100],Y_val[:100] ))

Tesing....
--------------------
Test acc: 0.99


# Saving Model and Weights
We can save the **model** in `json` and **weights** in a `hdf5` file format.

In [6]:
from keras.models import model_from_json

# serialize model to JSON
# the keras model which is trained is defined as 'model' in this example
model_json = model.to_json()
print('Saving Model Into JSON.')
with open("CapsuleNet/CapsuleTrained_Model/trained_model.json", "w") as json_file:
    json_file.write(model_json)

# previously we saved weight for each epochs. Weight of last epoch matched. 
# saving weights to HDF5
print('Saving Trained Model Weights Into HDF5.')
model.save_weights('CapsuleNet/CapsuleTrained_Model/trained_model_weights.h5')

Saving Model Into JSON.
Saving Trained Model Weights Into HDF5.


## Tensorboard
[Run Tensorboard](https://github.com/lspvic/jupyter_tensorboard) from jupyter notebook. To run from terminal: `tensorboard --logdir ./CapGraph`. In my case, I run this command from my working directory where a folder (Graph) has been created.

## Submission

In [17]:
# creating kaggle csv submission file
y_pred, _ = model.predict([test, 
                           np.zeros((data_test.shape[0],10))], 
                           batch_size = 32, 
                          verbose = True)    

with open('CapsuleNet/submission.csv', 'w') as out_file:
    out_file.write('ImageId,Label\n')
    for img_id, guess_label in enumerate(np.argmax(y_pred,1),1):
        out_file.write('%d,%d\n' % (img_id, guess_label))

28000/28000 [==============================] - 111s 4ms/step
